In [86]:
from typing import Optional

class RoshidError(Exception):
    #base error class for any errors inside of roshid
    pass

class RoshidAttributeError(RoshidError):
    def __init__(self, message):
        super().__init__(message)


In [82]:
from pydantic import BaseModel, ValidationError
from typing import Literal, Optional

class Attribute(BaseModel):
    attribute_name: str
    datatype: Literal['string', 'number', 'boolean']  # Restrict to specific values
    description: Optional[str] = None  # Description is optional



In [105]:
import json

class CustomerConfig:
    def __init__(self) -> None:
        self.attributes: set[Attribute] = []
        self.attributes.append(Attribute(attribute_name="Name", datatype="string", description="Name of the customer"))
        self.attributes.append(Attribute(attribute_name="Address", datatype="string", description="The delivery address mentioned by the customer"))
        self.attributes.append(Attribute(attribute_name="Phone", datatype="string", description="Phone number mentioned by the customer in the format: '01X XXXX XXXX'"))

    def generate_description(self) -> str:
        attribute_descriptions = [
            f"{attr.attribute_name} ({attr.datatype}): {attr.description or 'No description provided'}"
            for attr in self.attributes
        ]
        return f"Extract {', '.join(attribute_descriptions)} from the text:"
    
    def add_attribute(self, name:str, datatype: str, description: str):
        '''
        attribute_name: str = Name of the attribute (e.g 'Email', 'Secondary Contact')\n\n
        datatype: str ='string', 'number' or 'boolean'\n\n
        description: str = Description of the attribute for the LLM to accurately locate and extract the attribute
        '''
        try:
            if name.lower() in [attr.attribute_name.lower() for attr in self.attributes]: # checks if the attribute name already exists
                raise RoshidAttributeError("Attribute name already exists. Choose a different name.")
            else:
                self.attributes.append(Attribute(attribute_name=name, datatype=datatype, description=description))

        except ValidationError as e:
            print("Something went wrong when creating Attrbute.")
            print(e)

    def expected_json(self) -> str:
        """
        Returns a JSON schema that defines the structure of the expected output from the LLM.
        """
        schema = {}

        for attr in self.attributes:
            schema[attr.attribute_name] = {
                "type": attr.datatype,
                "description": attr.description,
                "value": ""
            }

        return json.dumps(schema)

In [106]:
cust1 = CustomerConfig()
cust1.add_attribute("Email", "string", "The email of the user")
cust1.add_attribute("is_purni", "boolean", "Checks if the users name is Purnima or anythis starting with the letter 'P'")
cust1.add_attribute("is_roy", "boolean", "Checks if the users name is roy")
cust1.add_attribute("product_count", "number", "Number of products ordered")
cust1.add_attribute("notes", "string", "If the customer mentions any note about the delivery, otherwise keep it empty.")

In [107]:
cust1.generate_description()
cust1.expected_json()

'{"Name": {"type": "string", "description": "Name of the customer", "value": ""}, "Address": {"type": "string", "description": "The delivery address mentioned by the customer", "value": ""}, "Phone": {"type": "string", "description": "Phone number mentioned by the customer in the format: \'01X XXXX XXXX\'", "value": ""}, "Email": {"type": "string", "description": "The email of the user", "value": ""}, "is_purni": {"type": "boolean", "description": "Checks if the users name is Purnima or anythis starting with the letter \'P\'", "value": ""}, "is_roy": {"type": "boolean", "description": "Checks if the users name is roy", "value": ""}, "product_count": {"type": "number", "description": "Number of products ordered", "value": ""}, "notes": {"type": "string", "description": "If the customer mentions any note about the delivery, otherwise keep it empty.", "value": ""}}'

In [91]:
from PIL import Image
import pytesseract

img = Image.open("../0.0/ss5.jpg")
pytesseract.image_to_string(img)

'139PMO @OEBO - will all BG)\n\n< Gi Uptown Panam & On D\n\nBusiness chat + uptow...\n\n=\n\nYou replied\n\nHow long would would take to deliver?\n\nCan | know your location?\n\n25 May 2021\n\n- Bashundhara r/a\n25 May 2021\n\nI’m very curious to know why are\nyou called bonbilai!\n\nFarhana Bilby\n\n01312221018\nhossain.farhanah@gmail.com\nHouse 11, road 333, block J,\nBashundhara r/a\n\n1x BTV F-shirt in L\n\n1x Gulistan circuit house tee in M\n\n- Please\n\n~\n\n- What is your payment system?\n\n©) vessave.. O A) ) @)\n'

In [93]:
GROQ_API_KEY="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"

'{\n  "properties": {\n    "attribute_name": {\n      "title": "Attribute Name",\n      "type": "string"\n    },\n    "datatype": {\n      "enum": [\n        "string",\n        "number",\n        "boolean"\n      ],\n      "title": "Datatype",\n      "type": "string"\n    },\n    "description": {\n      "anyOf": [\n        {\n          "type": "string"\n        },\n        {\n          "type": "null"\n        }\n      ],\n      "default": null,\n      "title": "Description"\n    }\n  },\n  "required": [\n    "attribute_name",\n    "datatype"\n  ],\n  "title": "Attribute",\n  "type": "object"\n}'

In [94]:
import json
dummy_json_schema = {
    "reciepient_name": "name of reciepient (str)",
    "reciepient_address": "address of reciepient (str)",
    "reciepient_phone": "reciepients phone number in the format 01X-XXXX-XXXX (str)",
    "items_ordered": "number of products the customer orderd",
    "is_roy": "customer has the surname 'Roy' (boolean)"
}



'{"reciepient_name": "name of reciepient (str)", "reciepient_address": "address of reciepient (str)", "reciepient_phone": "reciepients phone number in the format 01X-XXXX-XXXX (str)", "items_ordered": "number of products the customer orderd", "is_roy": "customer has the surname \'Roy\' (boolean)"}'

In [ ]:
%a, %b %-d - %-I:%M %p